In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
80152,Grant_Hanley,22,2,79,Norwich,Manchester United,0.316088,1.546205,False,0,...,0,4,0.0,2918,9581,6663,0,2020-01-11T15:00:00Z,1920,1.0
80153,Roberto_Pereyra,22,3,8,Watford,Bournemouth,0.488603,0.713194,False,6,...,3,0,17.0,-745,372,1117,0,2020-01-12T14:00:00Z,1920,1.0
80154,Sead_Kolasinac,22,2,90,Arsenal,Crystal Palace,1.416533,0.482482,False,2,...,1,1,6.0,4606,5911,1305,0,2020-01-11T12:30:00Z,1920,1.0
80155,Stuart_Armstrong,22,3,77,Southampton,Leicester City,0.470968,1.048266,False,9,...,2,1,25.0,13,184,171,1,2020-01-11T15:00:00Z,1920,1.0
80156,Rúben Diogo_da Silva Neves,22,3,90,Wolverhampton Wanderers,Newcastle United,0.703754,0.559829,True,2,...,1,1,3.0,-7613,5844,13457,0,2020-01-11T15:00:00Z,1920,1.0


In [5]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

23

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [7]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Andreas_Christensen,23,2,90,Chelsea,Newcastle United,1.743464,0.569576,False,2,...,42055,0,1,0.0,334,811,477,0,2020-01-18T17:30:00Z,1920
1,Kevin_De Bruyne,23,3,90,Manchester City,Crystal Palace,2.694208,0.481952,True,2,...,3898669,2,2,29.0,95576,133069,37493,0,2020-01-18T15:00:00Z,1920
2,Martin_Kelly,23,2,90,Crystal Palace,Manchester City,0.481952,2.694208,False,1,...,2222070,2,2,0.0,-83747,19514,103261,0,2020-01-18T15:00:00Z,1920
3,Gerard_Deulofeu,23,4,90,Watford,Tottenham Hotspur,0.488066,2.024801,True,2,...,203168,0,0,47.0,-1389,21005,22394,0,2020-01-18T12:30:00Z,1920
4,Christian_Atsu,23,3,0,Newcastle United,Chelsea,0.569576,1.743464,True,0,...,8806,0,1,0.0,-181,104,285,0,2020-01-18T17:30:00Z,1920
5,Gabriel Fernando_de Jesus,23,4,28,Manchester City,Crystal Palace,2.694208,0.481952,True,4,...,244561,2,2,14.0,38953,59183,20230,0,2020-01-18T15:00:00Z,1920
6,Adam_Webster,23,2,90,Brighton and Hove Albion,Aston Villa,0.445063,0.542779,True,1,...,57943,1,1,9.0,7493,12895,5402,1,2020-01-18T15:00:00Z,1920
7,Jacob_Murphy,23,3,0,Newcastle United,Chelsea,0.569576,1.743464,True,0,...,571,0,1,0.0,-5,9,14,0,2020-01-18T17:30:00Z,1920
8,Danny_Ings,23,4,90,Southampton,Wolverhampton Wanderers,0.470450,0.702152,True,2,...,2055116,3,2,22.0,495038,527509,32471,0,2020-01-18T15:00:00Z,1920
9,Alexis_Sánchez,23,3,0,Manchester United,Liverpool,1.544507,2.466237,False,0,...,28589,0,2,0.0,-68,32,100,0,2020-01-19T16:30:00Z,1920


In [8]:
df_latest.shape

(606, 34)

In [9]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [10]:
last_play_proba.head()

,player,play_proba
34,Shkodran_Mustafi,1.00
35,Héctor_Bellerín,0.25
36,Sead_Kolasinac,1.00
37,Ainsley_Maitland-Niles,1.00
38,Sokratis_Papastathopoulos,1.00


In [11]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [12]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
0,Andreas_Christensen,23,2,90,Chelsea,Newcastle United,1.743464,0.569576,False,2,...,0,1,0.0,334,811,477,0,2020-01-18T17:30:00Z,1920,1.00
1,Kevin_De Bruyne,23,3,90,Manchester City,Crystal Palace,2.694208,0.481952,True,2,...,2,2,29.0,95576,133069,37493,0,2020-01-18T15:00:00Z,1920,1.00
2,Martin_Kelly,23,2,90,Crystal Palace,Manchester City,0.481952,2.694208,False,1,...,2,2,0.0,-83747,19514,103261,0,2020-01-18T15:00:00Z,1920,1.00
3,Gerard_Deulofeu,23,4,90,Watford,Tottenham Hotspur,0.488066,2.024801,True,2,...,0,0,47.0,-1389,21005,22394,0,2020-01-18T12:30:00Z,1920,1.00
4,Christian_Atsu,23,3,0,Newcastle United,Chelsea,0.569576,1.743464,True,0,...,0,1,0.0,-181,104,285,0,2020-01-18T17:30:00Z,1920,1.00
5,Gabriel Fernando_de Jesus,23,4,28,Manchester City,Crystal Palace,2.694208,0.481952,True,4,...,2,2,14.0,38953,59183,20230,0,2020-01-18T15:00:00Z,1920,1.00
6,Adam_Webster,23,2,90,Brighton and Hove Albion,Aston Villa,0.445063,0.542779,True,1,...,1,1,9.0,7493,12895,5402,1,2020-01-18T15:00:00Z,1920,1.00
7,Jacob_Murphy,23,3,0,Newcastle United,Chelsea,0.569576,1.743464,True,0,...,0,1,0.0,-5,9,14,0,2020-01-18T17:30:00Z,1920,0.00
8,Danny_Ings,23,4,90,Southampton,Wolverhampton Wanderers,0.470450,0.702152,True,2,...,3,2,22.0,495038,527509,32471,0,2020-01-18T15:00:00Z,1920,1.00
9,Alexis_Sánchez,23,3,0,Manchester United,Liverpool,1.544507,2.466237,False,0,...,0,2,0.0,-68,32,100,0,2020-01-19T16:30:00Z,1920,0.00


In [13]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [14]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
80758,Sébastien_Haller,23,4,90,West Ham United,Everton,0.702049,0.996235,True,2,...,1,1,64.0,5972,15948,9976,0,2020-01-18T15:00:00Z,1920,1.0
80759,Riyad_Mahrez,23,3,17,Manchester City,Crystal Palace,2.694208,0.481952,True,1,...,2,2,8.0,285700,308158,22458,0,2020-01-18T15:00:00Z,1920,1.0
80760,Georges-Kévin_Nkoudou,23,3,0,Tottenham Hotspur,Watford,2.024801,0.488066,False,0,...,0,0,0.0,-7,8,15,0,2020-01-18T12:30:00Z,1920,0.0
80761,Sam_Byram,23,2,90,Norwich,Bournemouth,0.315740,0.712411,True,8,...,0,1,0.0,-55,209,264,0,2020-01-18T15:00:00Z,1920,1.0
80762,Harvey_Elliott,23,3,0,Liverpool,Manchester United,2.466237,1.544507,True,0,...,0,2,0.0,-6,47,53,0,2020-01-19T16:30:00Z,1920,1.0


In [15]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [16]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [17]:
fixtures.head(10)

,home_team,away_team,gw
179,West Ham United,Liverpool,24
230,Aston Villa,Watford,24
231,Bournemouth,Brighton and Hove Albion,24
232,Everton,Newcastle United,24
233,Leicester City,West Ham United,24
234,Sheffield United,Manchester City,24
235,Wolverhampton Wanderers,Liverpool,24
236,Manchester United,Burnley,24
237,Chelsea,Arsenal,24
238,Tottenham Hotspur,Norwich,24


In [18]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [19]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,24,West Ham United,Liverpool,21,14
1,24,Aston Villa,Watford,7,57
2,24,Bournemouth,Brighton and Hove Albion,91,36
3,24,Everton,Newcastle United,11,4
4,24,Leicester City,West Ham United,13,21


In [20]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [21]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [22]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,24,Liverpool,West Ham United,Andrew_Robertson,2,70,100,False
1,24,Liverpool,West Ham United,Trent_Alexander-Arnold,2,75,100,False
2,24,Liverpool,West Ham United,Virgil_van Dijk,2,64,100,False
3,24,Liverpool,West Ham United,Joseph_Gomez,2,52,100,False
4,24,Liverpool,West Ham United,Joel_Matip,2,52,100,False


In [23]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,24,West Ham United,Liverpool,Aaron_Cresswell,2,47,100,True
1,24,West Ham United,Liverpool,Angelo_Ogbonna,2,45,100,True
2,24,West Ham United,Liverpool,Arthur_Masuaku,2,42,100,True
3,24,West Ham United,Liverpool,Pablo_Zabaleta,2,42,None,True
4,24,West Ham United,Liverpool,Ryan_Fredericks,2,44,0,True


In [24]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 35.8,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 0.5,
 'Aaron_Mooy': 60.0,
 'Aaron_Ramsdale': 67.5,
 'Aaron_Wan-Bissaka': 72.0,
 'Abd-Al-Ali Morakinyo Olaposi_Koiki': 0.0,
 'Abdoulaye_Doucouré': 88.75,
 'Adam_Idah': 13.0,
 'Adam_Lallana': 25.8,
 'Adam_Masina': 40.4,
 'Adam_Smith': 90.0,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 78.4,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 81.5,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 54.0,
 'Ainsley_Maitland-Niles': 90.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 5.4,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 65.0,
 'Alexander_Tettey': 85.8,
 'Alexandre_Lacazette': 84.8,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 33.2,
 'Alisson_Ramses Becker': 90.0,
 'Andre_Gray': 2.2,
 'Andreas_Christensen': 36.0,
 'Andreas_Pereira': 67.4,
 'Andrew_Robertson': 89.4,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 0.0,
 'Andy_Carroll': 20.2,
 'Andy_Lonergan': 0.0,
 'Angel_Gomes': 2.0,
 'Angelo_Ogbo

In [25]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [26]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,24,West Ham United,Liverpool,Aaron_Cresswell,2,4.7,1.0,True,0.702049,2.466237,1920,90.000000
1,24,West Ham United,Liverpool,Angelo_Ogbonna,2,4.5,1.0,True,0.702049,2.466237,1920,72.000000
2,24,West Ham United,Liverpool,Arthur_Masuaku,2,4.2,1.0,True,0.702049,2.466237,1920,39.000000
3,24,West Ham United,Liverpool,Pablo_Zabaleta,2,4.2,1.0,True,0.702049,2.466237,1920,54.000000
4,24,West Ham United,Liverpool,Ryan_Fredericks,2,4.4,0.0,True,0.702049,2.466237,1920,0.000000
5,24,West Ham United,Liverpool,Issa_Diop,2,4.4,1.0,True,0.702049,2.466237,1920,36.000000
6,24,West Ham United,Liverpool,Fabián_Balbuena,2,4.3,1.0,True,0.702049,2.466237,1920,72.000000
7,24,West Ham United,Liverpool,Winston_Reid,2,3.9,1.0,True,0.702049,2.466237,1920,0.000000
8,24,West Ham United,Liverpool,Ben_Johnson,2,3.9,1.0,True,0.702049,2.466237,1920,0.000000
9,24,West Ham United,Liverpool,Marko_Arnautovic,4,7.0,0.0,True,0.702049,2.466237,1920,0.000000


In [27]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')